In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests
import logging

At the first block of every epoch
- Get the list of elected validators (elected_list)
- Get the stakes of validators who were not eligible for election in the last epoch (last block)
- Check if any of these validators present in elected_list

(note: election happen in the last block, however the epos-status changed in the first block)

In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("economic-test")
logger.setLevel(logging.INFO)

In [3]:
def get_information(method, params):
    url = 'https://api.s0.t.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [4]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [5]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [6]:
def getCurrentAndLastBlock():
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    return block, last_block

In [7]:
def getAllElectedValidator():
    method = "hmy_getElectedValidatorAddresses"
    params =[]
    return get_information(method, params)['result']

In [8]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [0]
    return get_information(method, params)['result'] 

In [9]:
def getEligibleValidator():
    eligible = []
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['epos-status'] == 'currently elected' or i['epos-status'] == 'eligible to be elected next epoch':
            address = i['validator']['address']
            eligible.append(address)
    return eligible

In [10]:
def E2_test(single):
    logger.info(f"Test-E2: Joining after the election start must not consider the validator")
    if single:
        curr_test = None
    else:
        curr_test = E3_test
    flag = True
    num = 1
    iterations = 0
    new_count = 0
    try:
        while iterations < num:
            # get the last block in current epoch
            block, last_block = getCurrentAndLastBlock()
            logger.info(f"current and last block numbers: {block}, {last_block}")
            logger.info(f"wait for the last second block ...")
            while block < last_block - 1:
                block = getBlockNumber()
            logger.info(f"last second block in the current epoch reached {block} will begin collecting existing eligible validators after 5 seconds")
            time.sleep(5)
            eligible_old = getEligibleValidator()

            while block < last_block:
                block = getBlockNumber()
            logger.info(f"last block in the current epoch reached {block} will begin collecting new eligible validators after 5 seconds")
            time.sleep(5)
            eligible_current = getEligibleValidator()

            logger.info(f"checking whether we have validators who set their status active after election starts")
            eligible_new = set(eligible_current) - set(eligible_old)
            if not eligible_new:
                logger.info(f"no validator joined after the election start in current test")
            else:
                new_count += 1
                while block < last_block + 1:
                    block = getBlockNumber()
                logger.info(f"first block in the current epoch reached {block} will wait for 5 seconds to begin collecting elected infos")
                time.sleep(3)
                logger.info(f"begin checking validators who joined after the election was elected...")
                validators = getAllValidatorInformation()
                for i in validators:
                    if i['validator']['address'] in eligible_new:
                        if i['currently-in-committee']:
                            logger.warning(f"Test-E2: Fail")
                            logger.warning(f"Validator  {i} joining after the election was considered for election\n")
                            flag = False
            iterations += 1
    except TypeError as e:
        logger.error(f"error: {e}")
    if new_count == 0:
        logger.info(f"Test-E2: No validator joined after the election in all tests, need more tests\n")
        return "Need More Tests", curr_test
    if flag:
        logger.info(f"Test-E2: Succeed\n")
        return True, curr_test
    else:
        return False, curr_test

In [11]:
E2_test(True)

INFO:economic-test:Test-E2: Joining after the election start must not consider the validator
INFO:economic-test:current and last block numbers: 3470276, 3473407
INFO:economic-test:wait for the last second block ...
INFO:economic-test:last second block in the current epoch reached 3473406 will begin collecting existing eligible validators after 5 seconds
INFO:economic-test:last block in the current epoch reached 3473407 will begin collecting new eligible validators after 5 seconds
INFO:economic-test:checking whether we have validators who set their status active after election starts
INFO:economic-test:no validator joined after the election start in current test
INFO:economic-test:Test-E2: No validator joined after the election in all tests, need more tests



('Need More Tests', None)